# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
spaceship_data = spaceship.copy()

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Selecting numerical columns for scaling

numerical_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# Handle missing values by imputing with the mean

imputer = SimpleImputer(strategy='mean')
spaceship_data[numerical_cols] = imputer.fit_transform(spaceship_data[numerical_cols])

# Initializing the scaler

scaler = StandardScaler()

# Scaling the numerical columns

spaceship_data[numerical_cols] = scaler.fit_transform(spaceship_data[numerical_cols])

# Display the first few rows to check the scaled values

spaceship_data[numerical_cols].head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0.709437,-0.340590,-0.287314,-0.290817,-0.276663,-0.269023
1,-0.336717,-0.175364,-0.281669,-0.248968,0.211505,-0.230194
2,2.034566,-0.275409,1.955616,-0.290817,5.694289,-0.225782
3,0.290975,-0.340590,0.517406,0.330225,2.683471,-0.098708
4,-0.894666,0.118709,-0.243409,-0.038048,0.225732,-0.267258


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

# Prepare the feature matrix and target vector

X = spaceship_data[numerical_cols]
y = spaceship_data['Transported'].astype(int)

# Initialize the logistic regression model for RFE

model = LogisticRegression()

# Initialize RFE with the logistic regression model

rfe = RFE(model, n_features_to_select=3)

# Fit RFE

fit = rfe.fit(X, y)

# Get the ranking of features

rfe_ranking = pd.DataFrame({
    'Feature': X.columns,
    'Ranking': fit.ranking_,
    'Selected': fit.support_
})

print("RFE Feature Ranking:\n", rfe_ranking)

# Initialize the Random Forest classifier for feature importance

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model

rf_model.fit(X, y)

# Get feature importances

importances = rf_model.feature_importances_

# Create a DataFrame for the feature importances

rf_feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print("Random Forest Feature Importances:\n", rf_feature_importances)

RFE Feature Ranking:
         Feature  Ranking  Selected
0           Age        4     False
1   RoomService        1      True
2     FoodCourt        2     False
3  ShoppingMall        3     False
4           Spa        1      True
5        VRDeck        1      True
Random Forest Feature Importances:
         Feature  Importance
4           Spa    0.203560
1   RoomService    0.202793
5        VRDeck    0.180226
2     FoodCourt    0.151314
0           Age    0.132271
3  ShoppingMall    0.129836


**Perform Train Test Split**

In [8]:
from sklearn.model_selection import train_test_split

# Use selected features from RFE and Random Forest

selected_features = ['RoomService', 'Spa', 'VRDeck']

# Split the data

X_train, X_test, y_train, y_test = train_test_split(spaceship_data[selected_features], y, test_size=0.2, random_state=42)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [9]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Initialize the BaggingClassifier

bagging_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1, random_state=42)

# Train the model

bagging_clf.fit(X_train, y_train)

# Predict

y_pred_bagging = bagging_clf.predict(X_test)

# Evaluate

bagging_accuracy = accuracy_score(y_test, y_pred_bagging)
print(f'Bagging Accuracy: {bagging_accuracy}')

Bagging Accuracy: 0.7688326624496837


- Random Forests

In [10]:
# Initialize the Random Forest classifier

rf_clf = RandomForestClassifier(n_estimators=500, random_state=42)

# Train the model

rf_clf.fit(X_train, y_train)

# Predict

y_pred_rf = rf_clf.predict(X_test)

# Evaluate

rf_accuracy = accuracy_score(y_test, y_pred_rf)
print(f'Random Forest Accuracy: {rf_accuracy}')

Random Forest Accuracy: 0.7435307648073606


- Gradient Boosting

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize the Gradient Boosting classifier

gb_clf = GradientBoostingClassifier(n_estimators=500, random_state=42)

# Train the model

gb_clf.fit(X_train, y_train)

# Predict

y_pred_gb = gb_clf.predict(X_test)

# Evaluate

gb_accuracy = accuracy_score(y_test, y_pred_gb)
print(f'Gradient Boosting Accuracy: {gb_accuracy}')

Gradient Boosting Accuracy: 0.757906843013226


- Adaptive Boosting

In [12]:
from sklearn.ensemble import AdaBoostClassifier

# Initialize the AdaBoost classifier

ada_clf = AdaBoostClassifier(n_estimators=500, random_state=42)

# Train the model

ada_clf.fit(X_train, y_train)

# Predict

y_pred_ada = ada_clf.predict(X_test)

# Evaluate

ada_accuracy = accuracy_score(y_test, y_pred_ada)
print(f'AdaBoost Accuracy: {ada_accuracy}')

/opt/anaconda3/envs/env_test/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Accuracy: 0.7573317998849913


Which model is the best and why?

In [13]:
model_accuracies = {
    'Bagging': bagging_accuracy,
    'Random Forest': rf_accuracy,
    'Gradient Boosting': gb_accuracy,
    'AdaBoost': ada_accuracy
}

best_model = max(model_accuracies, key=model_accuracies.get)
print(f'The best model is {best_model} with an accuracy of {model_accuracies[best_model]}')

The best model is Bagging with an accuracy of 0.7688326624496837
